In [4]:
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
from rasterio.windows import Window
import glob
from collections import defaultdict
from rasterio.merge import merge

In [6]:
dategroups = defaultdict(list)
image_dir ='chowan'
tif_files = glob.glob(os.path.join(image_dir, '*.tif'))
for tif_file in tif_files:
    filename = os.path.basename(tif_file)
    date = filename.split('_')[0]
#     print(date)
    dategroups[date].append(tif_file)

In [7]:
for date, files in dategroups.items():
    src_files_to_mosaic = []
    for tif_file in files:
        src = rasterio.open(tif_file)
        src_files_to_mosaic.append(src)
        
    mosaic, out_transform = merge(src_files_to_mosaic)
    
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform
    })
    output_path = f'mosaic_{date}.tif'
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
        
    for src in src_files_to_mosaic:
        src.close()

In [9]:
for mosaic_img in os.listdir('mosaic'):
    with rasterio.open(os.path.join('mosaic', mosaic_img)) as rds:
        data = rds.read()
        data = np.moveaxis(data, 0,2)
        img_arr = np.array(data[:,:,:3])
        print(img_arr.shape)
#         plt.imshow(img_arr)
#         plt.show()

(7074, 9594, 3)
(19767, 21153, 3)
(14158, 11537, 3)
(7040, 9557, 3)
(4582, 8992, 3)
(14955, 16395, 3)
(11706, 10613, 3)
(14025, 18748, 3)


KeyboardInterrupt: 